Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

#################
# My notebook issues
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
sys.path.append('home/egupta/.local/bin')

##############

import os
import numpy as np
import scipy.stats as ss
import networkx as nx
import csv 
import numpy as np
import seaborn as sns
from pandas import DataFrame
from scipy.optimize import curve_fit
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline

##qtensor imports
import qtensor
from qtensor import QAOA_energy
from qtensor import parameter_optimization as popt
import torch

2021-07-22 16:38:22,213- WARNING•	QuickBB solver is unavailable
2021-07-22 16:38:22,215- WARNING•	Tamaki solver is unavailable: No path /home/egupta/.local/lib/python3.7/site-packages/thirdparty/tamaki_treewidth


# Grabbing Graphs

In [2]:
resi = []
with open('20NodeRandomGraph1 (2).csv', 'r', newline = '') as g:
  spamreader = csv.reader(g, quotechar='|')
  for row in spamreader:
    resi.append(row)

#convert bunch of rows to matrices
n = int(len(resi)/20)
mat_list = []
for i in range(n):
  array = []
  for j in range(20):
    row = resi[i*20 +j]
    row = [float(k) for k in row]
    array.append(row)
  mat_list.append(array)

In [3]:
def mats_to_graphs(mat_list):
  """
  Given list of adjaceny matrix repr of graphs, converts to actual graphs
  """
  g_list = []
  for mat in mat_list:
    arr = np.array(mat)
    G = nx.convert_matrix.from_numpy_matrix(arr)
    g_list.append(G)
  return g_list

In [4]:
g_list = mats_to_graphs(mat_list)

In [5]:
len(g_list)

55

In [6]:
new_g_list = g_list[:28]

In [7]:
len(new_g_list)

28

# FindingLocal Optima

Old Function

In [8]:

def get_qaoa_params(graph, edge=None, gamma=[0.5], beta=[0.5], steps=50):
    loss_history, params_history = popt.qaoa_maxcut_torch(graph, gamma, beta,
                                                          ordering_algo='greedy',
                                                          Opt=torch.optim.RMSprop,
                                                          edge=edge,
                                                          opt_kwargs=dict(lr=0.002),
                                                          steps=steps,
                                                          pbar=False)
    best_ix = np.argmin(loss_history)
    return params_history[best_ix], loss_history[best_ix]

def get_edge_contribution(graph, edge, gamma, beta):
    sim = qtensor.QtreeSimulator()
    composer = qtensor.DefaultQAOAComposer(graph, gamma=gamma, beta=beta)
    composer.energy_expectation_lightcone(edge)
    return np.real(sim.simulate_batch(composer.circuit)[0])

def opt_runs(G, seeds = 20, steps = 200):
  """
  Doing multiple optimization runs, each with random initialized param
  Returning results which inlclude the optimal param and corresponding energies
  """

  #first do 20 optimization runs 
  s = 0
  g_results = []
    
  while s<seeds:
    print('seed #' + str(s))
    #initialize param
    gamma = [np.random.rand()]
    beta = [np.random.rand() ]
    initial_energy = QAOA_energy(G,gamma =  gamma, beta = beta)

    #optimization
    params, loss = get_qaoa_params(G, gamma=gamma, beta=beta,edge = None, steps=steps)
    #print('Finished get_qaoa_params')
    gamma_opt, beta_opt = params[0], params[1]
    opt_energy = QAOA_energy(G, gamma = gamma_opt, beta = beta_opt)

    result = np.asarray([float(gamma_opt), float(beta_opt), float(opt_energy), float(initial_energy)])
    g_results.append(result)

    # comparing with max
    s+=1

  
  return g_results

Execution Code

In [9]:
results = []
graph = 0
for g in new_g_list:
    graph+=1
    print(graph)
    result = opt_runs(G=g, seeds =20, steps = 200)
    results.append(result)

1
seed #0



seed #1



seed #2



seed #3



seed #4


...


seed #15



seed #16



seed #17



seed #18



seed #19


In [10]:
results

[[array([ 0.91261828,  0.61706263, 20.28474908, 16.23997611]),
  array([ 0.41260195,  0.38294721, 20.28474895, 14.73352041]),
  array([ 0.58738148,  0.11706122, 20.28474909, 14.9014795 ]),
  array([ 0.08740486,  0.88294667, 20.2847488 , 16.21342887]),
  array([-0.08738067,  0.61707199, 20.28474899, 15.49198384]),
  array([ 0.41261142,  0.38294563, 20.28474908, 14.07785715]),
  array([ 0.41261247,  0.88294476, 20.28474909, 16.31627147]),
  array([ 0.91261804,  0.61705536, 20.28474911, 17.54609243]),
  array([ 0.4126181 ,  0.38294446, 20.28474911, 15.85270556]),
  array([ 0.41261822,  0.38294446, 20.28474911, 19.62308202]),
  array([ 0.5873549 ,  0.61705792, 20.28474869, 14.63070819]),
  array([ 0.08738217,  0.38294467, 20.28474911, 14.36624904]),
  array([ 0.0873801 ,  0.88291776, 20.28474881, 13.4902597 ]),
  array([ 0.91261715,  0.61703134, 20.28474886, 17.00977118]),
  array([ 0.4126173 ,  0.38295364, 20.28474907, 10.7209172 ]),
  array([ 0.41261825,  0.38294443, 20.28474911, 15.2882

# Storing Data

In [11]:
with open('55_20NodeRandom_Local_Data_Part1.csv', 'w', newline = '') as f:
  thewriter = csv.writer(f)
  for g_results in results :
        for seed_results in g_results:
            thewriter.writerow(seed_results)